## T5 Question Answering Tutorial

Since we are familiar with the use of LLM, I will quickly go through the application.

In [11]:
# Install Required Libraries
# !pip install transformers torch

In [22]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

select the `llm-env` kernel before running the notebook

**Note**
install jupyter dependencies if needed before using this notebook
install torch based on their official website either in CPU or GPU based

In [23]:
# Load Pre-trained T5 Model for QA and Tokenizer
model_name = "t5-small"  # Options: t5-small, t5-base, t5-large, t5-3b, t5-11b


PegasusTokenizer requires the SentencePiece library. We can install using:
`python -m pip install sentencepiece`

In [24]:
# Initialize tokenizer and model
print("Loading tokenizer and model for QA...")
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
print("Tokenizer and model loaded successfully.")

Loading tokenizer and model for QA...


c:\Users\vatha\miniconda3\envs\llm-env\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vatha\.cache\huggingface\hub\models--t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is ex

Tokenizer and model loaded successfully.


In [25]:
# Example: Context and Question
context = (
    "The COVID-19 pandemic has had a profound impact on global health and the economy. "
    "Countries around the world have implemented measures to curb the spread of the virus, "
    "including lockdowns, social distancing, and vaccination campaigns. Despite these efforts, "
    "many challenges remain, including vaccine distribution, public compliance, and emerging variants."
)
question = "What measures have countries implemented to curb the spread of COVID-19?"


In [26]:
# Prepare Input for T5
input_text = f"question: {question} context: {context}"  # T5 expects input in this format
print("Input text for QA:", input_text)


Input text for QA: question: What measures have countries implemented to curb the spread of COVID-19? context: The COVID-19 pandemic has had a profound impact on global health and the economy. Countries around the world have implemented measures to curb the spread of the virus, including lockdowns, social distancing, and vaccination campaigns. Despite these efforts, many challenges remain, including vaccine distribution, public compliance, and emerging variants.


In [27]:
# Tokenize Input
print("Tokenizing input for QA...")
tokens = tokenizer(input_text, truncation=True, padding="longest", max_length=512, return_tensors="pt")
print("Tokens for QA:", tokens)

Tokenizing input for QA...
Tokens for QA: {'input_ids': tensor([[  822,    10,   363,  3629,    43,  1440,  6960,    12, 16110,     8,
          3060,    13,  2847,  7765,   308,  4481,    58,  2625,    10,    37,
          2847,  7765,   308,  4481,  2131,   221,  3113,    65,   141,     3,
             9, 13343,  1113,    30,  1252,   533,    11,     8,  2717,     5,
         30763,   300,     8,   296,    43,  6960,  3629,    12, 16110,     8,
          3060,    13,     8,  6722,     6,   379,  6081,  3035,     7,     6,
           569,  1028,    17,     9,  4733,     6,    11, 24639,  8203,     5,
             3,  4868,   175,  2231,     6,   186,  2428,  2367,     6,   379,
         12956,  3438,     6,   452,  5856,     6,    11,  7821,  6826,     7,
             5,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 

In [28]:
# Generate Answer
print("Generating answer...")
answer_ids = model.generate(tokens["input_ids"], max_length=50, num_beams=5, early_stopping=True)
print("Answer IDs:", answer_ids)


Generating answer...
Answer IDs: tensor([[    0,  6081,  3035,     7,     6,   569,  1028,    17,     9,  4733,
             6,    11, 24639,  8203,     1]])


In [29]:
# Decode Answer
answer = tokenizer.decode(answer_ids[0], skip_special_tokens=True)
print("Decoded answer:", answer)

Decoded answer: lockdowns, social distancing, and vaccination campaigns


In [30]:
# Display QA Pair
print("Question:\n", question)
print("\nContext:\n", context)
print("\nGenerated Answer:\n", answer)

Question:
 What measures have countries implemented to curb the spread of COVID-19?

Context:
 The COVID-19 pandemic has had a profound impact on global health and the economy. Countries around the world have implemented measures to curb the spread of the virus, including lockdowns, social distancing, and vaccination campaigns. Despite these efforts, many challenges remain, including vaccine distribution, public compliance, and emerging variants.

Generated Answer:
 lockdowns, social distancing, and vaccination campaigns


In [31]:
# Optional: Function for Answering Multiple Questions
def answer_questions(context, questions, model, tokenizer, max_length=50):
    answers = []
    for question in questions:
        input_text = f"question: {question} context: {context}"
        print("Processing question:", question)
        tokens = tokenizer(input_text, truncation=True, padding="longest", max_length=512, return_tensors="pt")
        print("Tokenized input:", tokens)
        outputs = model.generate(tokens["input_ids"], max_length=max_length, num_beams=5, early_stopping=True)
        print("Generated outputs:", outputs)
        answers.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
        print("Decoded answer:", answers[-1])
    return answers

In [32]:
# Example: Answering Multiple Questions
questions = [
    "What measures have countries implemented to curb the spread of COVID-19?",
    "What challenges remain despite these efforts?",
]

print("Answering multiple questions...")
answers = answer_questions(context, questions, model, tokenizer)
for i, answer in enumerate(answers):
    print(f"\nQuestion {i+1}:\n{questions[i]}\n\nAnswer {i+1}:\n{answer}")


Answering multiple questions...
Processing question: What measures have countries implemented to curb the spread of COVID-19?
Tokenized input: {'input_ids': tensor([[  822,    10,   363,  3629,    43,  1440,  6960,    12, 16110,     8,
          3060,    13,  2847,  7765,   308,  4481,    58,  2625,    10,    37,
          2847,  7765,   308,  4481,  2131,   221,  3113,    65,   141,     3,
             9, 13343,  1113,    30,  1252,   533,    11,     8,  2717,     5,
         30763,   300,     8,   296,    43,  6960,  3629,    12, 16110,     8,
          3060,    13,     8,  6722,     6,   379,  6081,  3035,     7,     6,
           569,  1028,    17,     9,  4733,     6,    11, 24639,  8203,     5,
             3,  4868,   175,  2231,     6,   186,  2428,  2367,     6,   379,
         12956,  3438,     6,   452,  5856,     6,    11,  7821,  6826,     7,
             5,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       

For the next step, we can try with different model and evaluate which one we prefer to use as a QA model.